In [9]:
#import libraries
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [11]:
# Read data
import json

# Open and read the JSON file
with open('Optimum.json', 'r') as file:
    data = json.load(file)

# # of donors - solution

In [14]:
# Create the graph
G = nx.DiGraph()

# Add nodes with attributes from the dataset
for idx, entry in enumerate(data):  # Replace 'data' with your dataset
    G.add_node(idx, Recipient=entry["Recipient"], Donor=entry["Donor"])

# Define donor-recipient compatibility
donor_recipient_compatibility = {
    'O': ['O', 'A', 'B', 'AB'], 
    'A': ['A', 'AB'], 
    'B': ['B', 'AB'], 
    'AB': ['AB']
}

# Add edges based on donor-recipient compatibility
for u in G.nodes:
    donors = G.nodes[u]["Donor"]
    for donor in donors:
        for v in G.nodes:
            if u == v:
                continue
            recipient = G.nodes[v]["Recipient"]
            if recipient in donor_recipient_compatibility.get(donor, []):
                G.add_edge(u, v)

# Define the optimization model
m = gp.Model()

# Add variables for each edge
x = m.addVars(G.edges, vtype=GRB.BINARY)

# Set the objective to maximize the number of selected edges
m.setObjective(gp.quicksum(x[e] for e in G.edges), GRB.MAXIMIZE)

# Add constraints for at most one incoming edge per node
m.addConstrs(gp.quicksum(x[(u, v)] for u in G.predecessors(v)) <= 1 for v in G.nodes)

# Add constraints for flow balance
m.addConstrs(gp.quicksum(x[(u, v)] for u in G.predecessors(v)) == gp.quicksum(x[(v, u)] for u in G.successors(v)) for v in G.nodes)

# Solve the model
m.optimize()

# Analyze the solution
if m.Status == GRB.OPTIMAL:
    # Count the number of willing donors for each recipient
    recipient_donor_data = [{"Recipient": i, "# of Donors": len(G.nodes[i]["Donor"])} for i in G.nodes]

    # Add match information based on the solution
    for entry in recipient_donor_data:
        recipient = entry["Recipient"]
        # Check if the recipient is matched in the solution
        entry["Matched"] = any(
            recipient == v for u, v in G.edges if x.get((u, v), 0).X > 0.5
        )

    # Convert to a DataFrame for analysis
    recipient_donor_df = pd.DataFrame(recipient_donor_data)

    # Group by the number of donors and summarize
    donor_analysis = recipient_donor_df.groupby("# of Donors").agg(
        **{
            "# of Recipients": ("Recipient", "size"),
            "# Matched": ("Matched", "sum")
        }
    ).reset_index()

    # Calculate the match percentage
    donor_analysis["Match Percentage"] = (donor_analysis["# Matched"] / donor_analysis["# of Recipients"]) * 100

    # Display the table
    print(donor_analysis)
else:
    print("No optimal solution found.")

Set parameter Username
Set parameter LicenseID to value 2589009
Academic license - for non-commercial use only - expires 2025-11-22
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.6.0 23G80)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1422 rows, 177611 columns and 532833 nonzeros
Model fingerprint: 0x64eeb380
Variable types: 0 continuous, 177611 integer (177611 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 658 rows and 104717 columns
Presolve time: 0.15s
Presolved: 764 rows, 72894 columns, 145788 nonzeros
Variable types: 0 continuous, 72894 integer (72894 binary)

Root relaxation: objective 3.520000e+02, 1468 iterations, 0.06 seconds (0.20 work units)

    Nodes    |    Current Node    |     Objective Bo

# blood type - solution

In [16]:
# Create the graph
G = nx.DiGraph()

# Add nodes with attributes from the dataset
for idx, entry in enumerate(data):  # Replace 'data' with your dataset
    G.add_node(idx, Recipient=entry["Recipient"], Donor=entry["Donor"])

# Define donor-recipient compatibility rules
donor_recipient_compatibility = {
    'O': ['O', 'A', 'B', 'AB'], 
    'A': ['A', 'AB'], 
    'B': ['B', 'AB'], 
    'AB': ['AB']
}

# Add edges based on donor-recipient compatibility
for u in G.nodes:
    donors = G.nodes[u]["Donor"]
    for donor in donors:
        for v in G.nodes:
            if u == v:
                continue
            recipient = G.nodes[v]["Recipient"]
            if recipient in donor_recipient_compatibility.get(donor, []):
                G.add_edge(u, v)

# Define the optimization model
m = gp.Model()

# Add variables for each edge
x = m.addVars(G.edges, vtype=GRB.BINARY)

# Set the objective to maximize the number of selected edges
m.setObjective(gp.quicksum(x[e] for e in G.edges), GRB.MAXIMIZE)

# Add constraints for at most one incoming edge per node
m.addConstrs(gp.quicksum(x[(u, v)] for u in G.predecessors(v)) <= 1 for v in G.nodes)

# Add constraints for flow balance
m.addConstrs(gp.quicksum(x[(u, v)] for u in G.predecessors(v)) == gp.quicksum(x[(v, u)] for u in G.successors(v)) for v in G.nodes)

# Solve the model
m.optimize()

# Analyze the solution based on recipient blood type
if m.Status == GRB.OPTIMAL:
    # Extract recipient blood types and match statuses
    recipient_data = [{"Recipient": i, "Blood Type": G.nodes[i]["Recipient"]} for i in G.nodes]

    # Add match information based on the solution
    for entry in recipient_data:
        recipient = entry["Recipient"]
        # Check if the recipient is matched in the solution
        entry["Matched"] = any(
            recipient == v for u, v in G.edges if x.get((u, v), 0).X > 0.5
        )

    # Convert to a DataFrame for analysis
    recipient_df = pd.DataFrame(recipient_data)

    # Group by blood type and summarize
    blood_type_analysis = recipient_df.groupby("Blood Type").agg(
        **{
            "# of Recipients": ("Recipient", "size"),
            "# Matched": ("Matched", "sum")
        }
    ).reset_index()

    # Calculate the match percentage
    blood_type_analysis["Match Percentage"] = (blood_type_analysis["# Matched"] / blood_type_analysis["# of Recipients"]) * 100

    # Display the table
    print(blood_type_analysis)
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.6.0 23G80)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1422 rows, 177611 columns and 532833 nonzeros
Model fingerprint: 0x64eeb380
Variable types: 0 continuous, 177611 integer (177611 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 658 rows and 104717 columns
Presolve time: 0.15s
Presolved: 764 rows, 72894 columns, 145788 nonzeros
Variable types: 0 continuous, 72894 integer (72894 binary)

Root relaxation: objective 3.520000e+02, 1468 iterations, 0.06 seconds (0.20 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     

# length of 3 cycles

In [17]:
# Read data and draw some figures
import json

# Open and read the JSON file
with open('Optimum.json', 'r') as file:
    data = json.load(file)

In [19]:
import networkx as nx

# Create a graph
G = nx.DiGraph()

# Create nodes
counter = 0
for element in data:
    #print(element)
    G.add_nodes_from([(counter, element)])
    counter += 1

#G.add_nodes_from(data)

#print(G.nodes)

donor_recipient_compatibility = {'O': ['O', 'A', 'B', 'AB'], 'A': ['A', 'AB'], 'B':['B', 'AB'], 'AB':['AB']}

# Create edges (relations)
for node in G.nodes:
    donors = G.nodes[node]['Donor']
    #print(donors)
    for donor in donors:
        for vertex in G.nodes:
            if node == vertex: continue
            recipient = G.nodes[vertex]['Recipient']
            if recipient not in donor_recipient_compatibility[donor]: continue
            G.add_edge(node, vertex)

In [21]:
# Find cycles of length 3

potential_cycle_3 = []

H = G.to_undirected()

for (u,v) in H.edges:
    for k in nx.common_neighbors(H, u, v):
        potential_cycle_3.append((u,v,k))
        
#print("potential_cycle_3:", potential_cycle_3)        
        
cycle_3 = [] 

for (u,v,k) in potential_cycle_3:
    if (u,v) in G.edges and (v,k) in G.edges and (k,u) in G.edges:
        cycle_3.append((u,v,k,u))
    if (v,u) in G.edges and (u,k) in G.edges and (k,v) in G.edges: 
        cycle_3.append((v,u,k,v))

print("length_cycle_3:", len(cycle_3))

length_cycle_3: 0


# length of 2 cycles

In [23]:
# Find cycles of length 2

cycle_2 = []

for (i,j) in G.edges:
    G.edges[(i,j)]["visited"] = False

for (i,j) in G.edges:
    if G.edges[(i,j)]["visited"] == True: continue
    if (j,i) in G.edges:
        cycle_2.append((i,j,i))
        G.edges[(j,i)]["visited"] = True
            
print("length_cycle_2:", len(cycle_2))

length_cycle_2: 36256
